In [1]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd
import numpy as np

# %load_ext nb_black
%load_ext lab_black

In [2]:
df_distances = pd.read_csv(
    "/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/distances.csv"
)
df_pairs = pd.read_csv(
    "/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/pairs.csv"
)

#### Data Prep

In [3]:
unique_c1 = list(df_distances.city1.unique())

# possible combinations
for c in unique_c1:
    d1 = [c, c, 0]
    df_distances = df_distances.append(
        pd.Series(d1, index=["city1", "city2", "distance"]), ignore_index=True
    )

/tmp/ipykernel_14500/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_14500/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_14500/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_14500/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_14500/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [4]:
df_pairs.loc[df_pairs.Dage == "-", "Dage"] = 99999999
df_pairs.Dage = df_pairs.Dage.astype(int)
df_pairs.loc[df_pairs.Rage == "-", "Rage"] = 99999999
df_pairs.Rage = df_pairs.Rage.astype(int)

df_pairs.loc[df_pairs.DBT == "-", "DBT"] = "NaN"
df_pairs.DBT = df_pairs.DBT.astype(str)
df_pairs.loc[df_pairs.RBT == "-", "RBT"] = "NaN"
df_pairs.RBT = df_pairs.RBT.astype(str)

In [5]:
# write the compatible bloodtypes
bloodtypes: Dict[str, int] = {
    # from A to everything
    ("A", "A"): 1,
    ("A", "B"): 0,
    ("A", "AB"): 1,
    ("A", "O"): 0,
    # from b to everything
    ("B", "A"): 0,
    ("B", "B"): 1,
    ("B", "AB"): 1,
    ("B", "O"): 0,
    # from AB to everything
    ("AB", "A"): 0,
    ("AB", "B"): 0,
    ("AB", "AB"): 1,
    ("AB", "O"): 0,
    # from O to everything
    ("O", "A"): 1,
    ("O", "B"): 1,
    ("O", "AB"): 1,
    ("O", "O"): 1,
    # missing blootypes
    ("NaN", "A"): 0,
    ("NaN", "B"): 0,
    ("NaN", "AB"): 0,
    ("NaN", "O"): 0,
    ("A", "NaN"): 0,
    ("B", "NaN"): 0,
    ("AB", "NaN"): 0,
    ("O", "NaN"): 0,
}


# ID = df_pairs["ID"]
# rbt = df_pairs["RBT"]

In [6]:
# change the order of the df to be more intuitive
RBT = df_pairs.loc[:, "RBT"]
DBT = df_pairs.loc[:, "DBT"]

# drop column
df_pairs.drop(columns=["RBT", "DBT"], axis=1, inplace=True)

# add them back
df_pairs["DBT"] = DBT
df_pairs["RBT"] = RBT

df_pairs.loc[
    (df_pairs.type == "pair")
    & (df_pairs.Rage.astype(int) - df_pairs.Dage.astype(int) <= 10)
    & (df_pairs.Location == 1)
    & (bloodtypes[(df_pairs.DBT, df_pairs.RBT)] == 1),
    "C",
] = 1

In [7]:
df_pairs["C"] = np.nan

In [8]:
for i, r in df_pairs.iterrows():
    if (
        (df_pairs.iloc[i].type == "pair")
        & (df_pairs.iloc[i].Rage.astype(int) - df_pairs.iloc[i].Dage.astype(int) <= 10)
        & (bloodtypes[(df_pairs.iloc[i].DBT, df_pairs.iloc[i].RBT)] == 1)
    ) == True:
        df_pairs.loc[i, "C"] = 1
    else:
        df_pairs.loc[i, "C"] = 0

1. Match a donor with a recipient where bloot type works
2. 

In [9]:
df_pairs

,ID,type,Rage,Dage,Location,DBT,RBT,C
0,1,receiver,58,99999999,Paris,NaN,O,0.0
1,2,pair,33,35,Moscow,B,A,0.0
2,3,receiver,69,99999999,Istanbul,NaN,O,0.0
3,4,pair,30,69,Frankfurt,A,O,0.0
4,5,receiver,62,99999999,Istanbul,NaN,A,0.0
...,...,...,...,...,...,...,...,...
195,196,pair,54,51,Lisbon,O,O,1.0
196,197,pair,57,34,Manchester,B,A,0.0
197,198,receiver,68,99999999,Madrid,NaN,A,0.0
198,199,pair,49,49,Copenhagen,O,A,1.0


### create arcs

for i, r in df_pairs.iterrows():
    if (
        (df_pairs.iloc[i].type == "pair")
        & (df_pairs.iloc[i].Rage.astype(int) - df_pairs.iloc[i].Dage.astype(int) <= 10)
        & (bloodtypes[(df_pairs.iloc[i].DBT, df_pairs.iloc[i].RBT)] == 1)
    ) == True:
        df_pairs.loc[i, "C"] = 1
    else:
        df_pairs.loc[i, "C"] = 0

In [18]:
df_pairs

,ID,type,Rage,Dage,Location,DBT,RBT,C
0,1,receiver,58,99999999,Paris,NaN,O,0.0
1,2,pair,33,35,Moscow,B,A,0.0
2,3,receiver,69,99999999,Istanbul,NaN,O,0.0
3,4,pair,30,69,Frankfurt,A,O,0.0
4,5,receiver,62,99999999,Istanbul,NaN,A,0.0
...,...,...,...,...,...,...,...,...
195,196,pair,54,51,Lisbon,O,O,1.0
196,197,pair,57,34,Manchester,B,A,0.0
197,198,receiver,68,99999999,Madrid,NaN,A,0.0
198,199,pair,49,49,Copenhagen,O,A,1.0


In [19]:
df_distances

,city1,city2,distance
0,Amsterdam,London,222
1,London,Frankfurt,397
2,London,Amsterdam,222
3,Paris,Amsterdam,267
4,Frankfurt,Paris,297
...,...,...,...
319,Manchester,Manchester,0
320,Milan,Milan,0
321,Palma de Mallorca,Palma de Mallorca,0
322,Lisbon,Lisbon,0


In [15]:
arcs_: Dict[tuple, list] = {}

for i, r in df_pairs.iterrows():
    id_ = df_pairs.iloc[i].ID
    type_ = df_pairs.iloc[i].type
    Rage_ = df_pairs.iloc[i].Rage
    Dage_ = df_pairs.iloc[i].Dage
    location_ = df_pairs.iloc[i].Location
    compatible_ = df_pairs.iloc[i].C
    RBT_ = df_pairs.iloc[i].RBT
    DBT_ = df_pairs.iloc[i].DBT

    break


    # pairs 
    if type_ == "pair":
        # grap the city and only select those rows in "city1" where  the df_distances.city.astye(str) == location_
        # create a subset of cities that are valid 
        possible_cities1 = list(df_distances[df_distances.city.astye(str) == location_ & df_distances.distance <= 300].city2)
        if ((df_pairs.iloc[i].type == "pair")
             & (df_pairs.iloc[i].Rage.astype(int) - df_pairs.iloc[i].Dage.astype(int) <= 10)
                & (bloodtypes[(df_pairs.iloc[i].DBT, df_pairs.iloc[i].RBT)] == 1)
                 ) == True:
                 



        # test all conditions
        if 
            arcs_[("pair",id_)] = 







In [17]:
# if type == pairs
df_pairs[
    (
        (df_pairs.Rage - Rage_ <= 10)
        & (df_pairs.Location == location_)
        & (df_pairs.C == compatible_)
    )
]

# elif type receiver
df_pairs[
    (
        ((df_pairs.type == "pair") |(df_pairs.type == "donor") )

        & (df_pairs.Rage == Rage_)
        & (df_pairs.Location == location_)
        & (df_pairs.C == compatible_)
        & (df_pairs.RBT == RBT_)
        & (df_pairs.DBT == DBT_)
    )
]


# elif type donor

,ID,type,Rage,Dage,Location,DBT,RBT,C
0,1,receiver,58,99999999,Paris,NaN,O,0.0


In [10]:
df_pairs.iloc[i].type

'pair'

In [11]:
df_pairs

,ID,type,Rage,Dage,Location,DBT,RBT,C
0,1,receiver,58,99999999,Paris,NaN,O,0.0
1,2,pair,33,35,Moscow,B,A,0.0
2,3,receiver,69,99999999,Istanbul,NaN,O,0.0
3,4,pair,30,69,Frankfurt,A,O,0.0
4,5,receiver,62,99999999,Istanbul,NaN,A,0.0
...,...,...,...,...,...,...,...,...
195,196,pair,54,51,Lisbon,O,O,1.0
196,197,pair,57,34,Manchester,B,A,0.0
197,198,receiver,68,99999999,Madrid,NaN,A,0.0
198,199,pair,49,49,Copenhagen,O,A,1.0


when pair, then if you give away you have to receive one!!!
#


you can only give one and receive one!



note: we should also include the Pairs into a possible combination as they are definitiely also themselfes